<a href="https://colab.research.google.com/github/fjgr/IA_BigData/blob/main/M2D/Tarea-3/M2D_Tarea_3_Planificaci%C3%B3n_STRIPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# TAREA: Implementación de STRIPS para el Mundo de Bloques
# --------------------------------------------------------

"""
1. DEFINICIÓN DE ESTADOS
-----------------------
En este problema del mundo de bloques, utilizamos un vector de 24 bits para representar los estados:
- Los primeros 12 bits representan la presencia de bloques (1) o ausencia (0)
- Los últimos 12 bits representan la presencia de huecos (1) o ausencia (0)
- El tablero es de 4x3, donde cada columna puede contener hasta 4 bloques
"""

class BlocksWorld:
    def __init__(self):
        self.rows = 4
        self.cols = 3
        self.total_cells = self.rows * self.cols

        # 1.1 Estado Inicial: BACD en columna 0
        self.initial_state = [0] * (self.total_cells * 2)
        self.initial_state[0] = 1  # B en (0,0)
        self.initial_state[3] = 1  # A en (1,0)
        self.initial_state[6] = 1  # C en (2,0)
        self.initial_state[9] = 1  # D en (3,0)

        # Inicializar huecos complementarios
        for i in range(self.total_cells):
            if self.initial_state[i] == 0:
                self.initial_state[i + self.total_cells] = 1

        # 1.2 Estado Objetivo: ABCD en columna 2
        self.goal_state = [0] * (self.total_cells * 2)
        self.goal_state[2] = 1   # A en (0,2)
        self.goal_state[5] = 1   # B en (1,2)
        self.goal_state[8] = 1   # C en (2,2)
        self.goal_state[11] = 1  # D en (3,2)

        # Inicializar huecos complementarios para el estado objetivo
        for i in range(self.total_cells):
            if self.goal_state[i] == 0:
                self.goal_state[i + self.total_cells] = 1

        # Rastreo de posiciones de bloques
        self.block_positions = {
            'B': (0, 0),
            'A': (1, 0),
            'C': (2, 0),
            'D': (3, 0)
        }

    """
    2. IMPLEMENTACIÓN DE ACCIONES
    ----------------------------
    La única acción posible es mover(origen, destino), pero está sujeta a restricciones:
    - Solo se puede mover un bloque si no tiene otros bloques encima
    - El bloque debe moverse a una posición con soporte (otro bloque o el suelo)
    - Debe haber un hueco en la posición destino
    """

    def get_preconditions(self, from_pos, to_pos):
        """
        Verifica las precondiciones necesarias para realizar un movimiento
        """
        PC = [0] * (self.total_cells * 2)
        from_row, from_col = from_pos
        to_row, to_col = to_pos

        # Precondiciones para la posición origen
        # a) No debe haber bloques encima
        for row in range(from_row):
            idx = self.get_index(row, from_col) + self.total_cells
            PC[idx] = 1  # Necesitamos huecos arriba

        # b) Debe haber un bloque en la posición origen
        PC[self.get_index(from_row, from_col)] = 1

        # Precondiciones para la posición destino
        # a) Debe haber un hueco en la posición destino
        PC[self.get_index(to_row, to_col) + self.total_cells] = 1

        # b) Si no es el suelo, debe haber soporte debajo
        if to_row < self.rows - 1:
            PC[self.get_index(to_row + 1, to_col)] = 1

        return PC

    def check_preconditions(self, from_pos, to_pos, state):
        """
        Verifica si se cumplen las precondiciones para un movimiento
        """
        PC = self.get_preconditions(from_pos, to_pos)

        # Verificar que se cumplen todas las precondiciones
        for i in range(len(PC)):
            if PC[i] == 1 and state[i] == 0:
                return False, "No se cumplen las precondiciones"

        return True, "Movimiento válido"

    """
    3. FUNCIÓN DE TRANSICIÓN
    -----------------------
    La función de transición actualiza el estado del mundo después de realizar una acción.
    Se implementa mediante los efectos (add list y delete list) de STRIPS.
    """

    def get_effects(self, from_pos, to_pos):
        """
        Genera las listas de añadir y eliminar para un movimiento
        """
        add_list = [0] * (self.total_cells * 2)
        delete_list = [0] * (self.total_cells * 2)

        from_idx = self.get_index(from_pos[0], from_pos[1])
        to_idx = self.get_index(to_pos[0], to_pos[1])

        # Efectos en origen
        delete_list[from_idx] = 1  # Eliminar bloque
        add_list[from_idx + self.total_cells] = 1  # Añadir hueco

        # Efectos en destino
        add_list[to_idx] = 1  # Añadir bloque
        delete_list[to_idx + self.total_cells] = 1  # Eliminar hueco

        return add_list, delete_list

    def apply_effects(self, state, add_list, delete_list):
        """
        Aplica los efectos a un estado para generar el nuevo estado
        """
        new_state = state.copy()
        for i in range(len(state)):
            if add_list[i] == 1:
                new_state[i] = 1
            if delete_list[i] == 1:
                new_state[i] = 0
        return new_state

    """
    4. PLANIFICACIÓN CON STRIPS
    --------------------------
    La secuencia de acciones para llegar del estado inicial al objetivo
    se ha determinado analizando las precondiciones y efectos de cada movimiento.
    """

    def get_plan(self):
        """
        Retorna el plan de acciones para llegar al estado objetivo
        """
        return [
            ('B', (0,0), (3,1)),  # B: (0,0) → (3,1)
            ('A', (1,0), (2,1)),  # A: (1,0) → (2,1)
            ('C', (2,0), (1,1)),  # C: (2,0) → (1,1)
            ('D', (3,0), (3,2)),  # D: (3,0) → (3,2)
            ('C', (1,1), (2,2)),  # C: (1,1) → (2,2)
            ('A', (2,1), (3,0)),  # A: (2,1) → (3,0)
            ('B', (3,1), (1,2)),  # B: (3,1) → (1,2)
            ('A', (3,0), (0,2))   # A: (3,0) → (0,2)
        ]

    """
    5. EJECUCIÓN Y VISUALIZACIÓN
    ---------------------------
    Funciones para ejecutar el plan y visualizar los resultados
    """

    def get_index(self, row, col):
        """Convierte coordenadas a índice"""
        return row * self.cols + col

    def print_vector(self, vector, title):
        """Imprime un vector de estado de forma legible"""
        print(f"\n{title}:")
        print("Bloques:", " ".join(map(str, vector[:12])))
        print("Huecos: ", " ".join(map(str, vector[12:])))

    def print_state_grid(self, state):
        """Imprime el estado como una cuadrícula"""
        print("\nEstado del tablero:")
        for row in range(self.rows):
            line = ""
            for col in range(self.cols):
                block_found = False
                for block, pos in self.block_positions.items():
                    if pos == (row, col):
                        line += f" {block} "
                        block_found = True
                        break
                if not block_found:
                    line += " · "
            print(line)

    def execute_plan(self):
        """
        Ejecuta el plan completo mostrando cada paso
        """
        current_state = self.initial_state
        print("\n=== ESTADO INICIAL ===")
        self.print_state_grid(current_state)
        self.print_vector(current_state, "Vector de estado inicial")

        for i, (block, from_pos, to_pos) in enumerate(self.get_plan(), 1):
            print(f"\n{'='*50}")
            print(f"=== PASO {i}: Mover {block} de {from_pos} a {to_pos} ===")

            # Mostrar precondiciones
            PC = self.get_preconditions(from_pos, to_pos)
            self.print_vector(PC, "Precondiciones (PC)")

            # Mostrar efectos
            add_list, delete_list = self.get_effects(from_pos, to_pos)
            self.print_vector(add_list, "Add list")
            self.print_vector(delete_list, "Delete list")

            # Aplicar efectos
            current_state = self.apply_effects(current_state, add_list, delete_list)
            self.block_positions[block] = to_pos

            # Mostrar nuevo estado
            print("\n--- Resultado del movimiento ---")
            self.print_state_grid(current_state)
            self.print_vector(current_state, "Nuevo vector de estado")

# Ejecutar la solución
if __name__ == "__main__":
    world = BlocksWorld()
    world.execute_plan()


=== ESTADO INICIAL ===

Estado del tablero:
 B  ·  · 
 A  ·  · 
 C  ·  · 
 D  ·  · 

Vector de estado inicial:
Bloques: 1 0 0 1 0 0 1 0 0 1 0 0
Huecos:  0 1 1 0 1 1 0 1 1 0 1 1

=== PASO 1: Mover B de (0, 0) a (3, 1) ===

Precondiciones (PC):
Bloques: 1 0 0 0 0 0 0 0 0 0 0 0
Huecos:  0 0 0 0 0 0 0 0 0 0 1 0

Add list:
Bloques: 0 0 0 0 0 0 0 0 0 0 1 0
Huecos:  1 0 0 0 0 0 0 0 0 0 0 0

Delete list:
Bloques: 1 0 0 0 0 0 0 0 0 0 0 0
Huecos:  0 0 0 0 0 0 0 0 0 0 1 0

--- Resultado del movimiento ---

Estado del tablero:
 ·  ·  · 
 A  ·  · 
 C  ·  · 
 D  B  · 

Nuevo vector de estado:
Bloques: 0 0 0 1 0 0 1 0 0 1 1 0
Huecos:  1 1 1 0 1 1 0 1 1 0 0 1

=== PASO 2: Mover A de (1, 0) a (2, 1) ===

Precondiciones (PC):
Bloques: 0 0 0 1 0 0 0 0 0 0 1 0
Huecos:  1 0 0 0 0 0 0 1 0 0 0 0

Add list:
Bloques: 0 0 0 0 0 0 0 1 0 0 0 0
Huecos:  0 0 0 1 0 0 0 0 0 0 0 0

Delete list:
Bloques: 0 0 0 1 0 0 0 0 0 0 0 0
Huecos:  0 0 0 0 0 0 0 1 0 0 0 0

--- Resultado del movimiento ---

Estado del tablero:
 · 